In [ ]:
max_match_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_match_ts", debugValue="2026-01-01 00:00:00")
max_delivery_ts = dbutils.jobs.taskValues.get(taskKey="Extract_Max_ts", key="max_delivery_ts", debugValue="2026-01-01 00:00:00")

print(max_match_ts, max_delivery_ts)

In [ ]:
# Read the table
df = spark.sql(f"SELECT * FROM bronze.match_loading WHERE ingestion_ts <= '{max_match_ts}'")

In [ ]:
umpire_df = df.select(["umpire1","umpire2","umpire3"])

In [ ]:
from pyspark.sql.functions import col, lit, current_timestamp

# .distinct() ensures we don't try to insert the same umpire twice from the same batch
source_umpire_df = umpire_df.select(col("umpire1").alias("umpire_name")) \
    .union(umpire_df.select(col("umpire2").alias("umpire_name"))) \
    .union(umpire_df.select(col("umpire3").alias("umpire_name"))) \
    .filter(col("umpire_name").isNotNull()) \
    .distinct()

In [ ]:
# 2. Prepare Final DataFrame: Add the missing schema columns
final_umpire_df = source_umpire_df.select(
    col("umpire_name"),
    lit(None).cast("date").alias("date_of_birth"),
    lit(None).cast("string").alias("country"),
    current_timestamp().alias("created_at")
)



In [ ]:
# 3. Create Temp View for SQL
final_umpire_df.createOrReplaceTempView("source_new_umpires")


In [ ]:
# 4. Perform MERGE (Upsert)
spark.sql("""
    MERGE INTO silver.dim_umpire AS target
    USING source_new_umpires AS source
    ON target.umpire_name = source.umpire_name
    
    WHEN NOT MATCHED THEN
      INSERT (
        umpire_name, 
        date_of_birth, 
        country, 
        created_at
      )
      VALUES (
        source.umpire_name, 
        source.date_of_birth, 
        source.country, 
        source.created_at
      )
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]